In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import torch 
import json
import sys
import torch
import torch.nn.functional as F
import os
import numpy as np

sys.path.append("src")
os.chdir('/home/george-vengrovski/Documents/projects/tweety_bert_paper')

In [2]:
from torch.utils.data import DataLoader
from data_class import SongDataSet_Image, CollateFunction

train_dir = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/llb3_train_1_precent"
test_dir = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/llb3_test"

train_dataset = SongDataSet_Image(train_dir, num_classes=21)
test_dataset = SongDataSet_Image(test_dir, num_classes=21)

collate_fn = CollateFunction(segment_length=1000)  # Adjust the segment length if needed

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [3]:
from linear_probe import LinearProbeModel, LinearProbeTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier_model = LinearProbeModel(num_classes=21, model_type="raw", model=None, freeze_layers=None, layer_num=None, layer_id=None, classifier_dims=196)
classifier_model = classifier_model.to(device)

In [4]:
trainer = LinearProbeTrainer(model=classifier_model, train_loader=train_loader, test_loader=test_loader, device=device, lr=1e-3, plotting=True, batches_per_eval=100, desired_total_batches=1e4, patience=4)
trainer.train()

Batch 100: FER = 57.68%, Train Loss = 2.1114, Val Loss = 2.3344
Batch 200: FER = 54.12%, Train Loss = 1.8949, Val Loss = 2.0793


KeyboardInterrupt: 

In [5]:
from linear_probe import ModelEvaluator


# Initialize the ModelEvaluator with the filter_unseen_classes feature
evaluator = ModelEvaluator(model=classifier_model, 
                           test_loader=test_loader, 
                           num_classes=21,  # Assuming there are 21 possible classes
                           device='cuda:0',  # Use CUDA if available
                           use_tqdm=True,  # Enable a progress bar for the evaluation
                           filter_unseen_classes=True,  # Enable filtering based on training set classes
                           train_dir=train_dir)  # Path to the training dataset directory

# Perform model validation with multiple passes
class_frame_error_rates, total_frame_error_rate = evaluator.validate_model_multiple_passes(num_passes=1, max_batches=1250)

# Save the results to a specified directory, for example 'evaluation_results'
evaluator.save_results(class_frame_error_rates, total_frame_error_rate, '')

Evaluating:   0%|          | 0/142 [00:00<?, ?batch/s]

Evaluating: 100%|██████████| 142/142 [00:05<00:00, 27.74batch/s]
